In [2]:
%pip install selenium webdriver-manager

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.7 MB 1.7 MB/s eta 0:00:06
   --- ------------------------------------ 0.8/9.7 MB 1.7 MB/s eta 0:00:06
   ----- ---------------------------------- 1.3/9.7 MB 1.9 MB/s eta 0:00:05
   ------- -------------------------------- 1.8/9.7 MB 2.0 MB/s eta 0:00:04
   --------- ------------------------------ 2.4/9.7 MB 2.1 MB/s eta 0:00:04
   ----------- ---------------------------- 2.9/9.7 MB 2.2 MB/s eta 0:00:04
   --------------- ------------------------ 3.7/9.7 MB 2.4 MB/s eta 0:00:03
   ------------------ --------------------- 4.5/9.7 MB 2.5 MB/s eta 0:00:03
   -------------------- ------------------- 5.0/9.7 MB 2.6 MB/s eta 0:00:02
   ------------------------ --------------- 6.0/9.7 MB 2.8 MB/s eta 0:00:02
   ----------------------------- ---------- 7.1/9.7 MB 3.0 MB/s eta 0:00:01
   -----------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
httpcore 0.9.1 requires h11<0.10,>=0.8, but you have h11 0.16.0 which is incompatible.
langsmith 0.1.135 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
openai 1.51.2 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
spyder 5.5.1 requires ipython!=8.17.1,<9.0.0,>=8.13.0; python_version > "3.8", but you have ipython 9.0.2 which is incompatible.
streamlit 1.32.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.
tensorflow-intel 2.16.1 requires ml-dtypes~=0.3.1, but you have ml-dtypes 0.5.1 which is incompatible.
tensorflow-intel 2.16.1 requires tensorboard<2.17,>=2.16, but you have tensorboard 2.19.0 which is incompatible.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time

# Dictionnaire des URLs
crypto_urls = {
    'BTC-USD': 'https://finance.yahoo.com/quote/BTC-USD/history/',
    'HYPER36428-USD': 'https://finance.yahoo.com/quote/HYPER36428-USD/history/',
    'HOOD37295-USD': 'https://finance.yahoo.com/quote/HOOD37295-USD/history/'
}

# Dictionnaire pour stocker les DataFrames
crypto_dataframes = {}

options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


for ticker, url in crypto_urls.items():
    
    
    
    driver.get(url)
    time.sleep(15)  
    
    # Parser le HTML
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    
    table = soup.find("table", class_="table yf-1mr6o9m noDl hideOnPrint")
    
    if table:
        rows = table.find_all("tr")
        data_list = []
        
        for row in rows[1:]:  
            cols = row.find_all("td")
            if len(cols) >= 6:
                date_str = cols[0].text.strip()
                try:
                    date_obj = datetime.strptime(date_str, "%b %d, %Y")
                   
                    if date_obj.month in [9, 10, 11]:
                        open_price = cols[1].text.strip()
                        high = cols[2].text.strip()
                        low = cols[3].text.strip()
                        close = cols[4].text.strip()
                        adj_close = cols[5].text.strip()
                        volume = cols[6].text.strip() if len(cols) > 6 else '0'
                        
                        data_list.append({
                            'Date': date_str,
                            'Open': open_price,
                            'High': high,
                            'Low': low,
                            'Close': close,
                            'Adj Close': adj_close,
                            'Volume': volume
                        })
                except ValueError:
                    continue
        
       
        df = pd.DataFrame(data_list)
        crypto_dataframes[ticker] = df
        
        print(f"{ticker}: {len(df)} lignes scrapées")
        print(df.head(3))
        
    else:
        print(f"Table non trouvée pour {ticker}")


driver.quit()


if crypto_dataframes:
    for ticker, df in crypto_dataframes.items():
        df['Ticker'] = ticker
    
    
    df_all = pd.concat(crypto_dataframes.values(), ignore_index=True)
    
    print(f"\n=== RÉSULTAT FINAL ===")
    print(f"Total de lignes : {len(df_all)}")
    print(f"Cryptos : {list(crypto_dataframes.keys())}")
    
    
    df_all.to_csv('données_de_lhistorique_des_crypto.csv', index=False)
    print("Données sauvegardées")
    
else:
    print(" Aucune donnée n'a pu être scrapée")

BTC-USD: 91 lignes scrapées
           Date       Open       High        Low      Close  Adj Close  \
0  Nov 30, 2025  90,838.21  91,965.05  90,394.31  90,394.31  90,394.31   
1  Nov 29, 2025  90,918.74  91,187.62  90,260.19  90,851.76  90,851.76   
2  Nov 28, 2025  91,285.38  92,969.09  90,257.12  90,919.27  90,919.27   

           Volume  
0  38,497,902,869  
1  37,921,773,455  
2  60,895,830,289  
HYPER36428-USD: 91 lignes scrapées
           Date      Open      High       Low     Close Adj Close   Volume
0  Nov 30, 2025  0.003043  0.003205  0.000031  0.000031  0.000031  276,673
1  Nov 29, 2025  0.003182  0.003191  0.002837  0.003043  0.003043   22,405
2  Nov 28, 2025  0.003067  0.003249  0.002900  0.003182  0.003182   21,474
HOOD37295-USD: 91 lignes scrapées
           Date      Open      High       Low     Close Adj Close   Volume
0  Nov 30, 2025  0.081214  1.401900  0.001620  0.072958  0.072958        -
1  Nov 29, 2025  0.001071  0.676117  0.000034  0.081214  0.081214        -
2